In [ ]:
import pandas as pd
import time

In [ ]:
cards = pd.read_csv("csv/cards.csv")
translate = pd.read_csv("csv/foreign_data.csv")

In [ ]:
#Portuguese (Brazil), Korean, Japanese, French, German, Chinese Traditional, Chinese Simplified
esp = translate[translate['language']=="Spanish"] 
esp.info()

In [ ]:
print(len(esp)/len(cards))

### Atención:

* en español, hay 33969 cartas, que corresponde a un 52.67% del universo de cartas oficiales en inglés.

### Objetivo 1: 

* reemplazar entradas 'nan' del campo *type* por su texto en inglés.
> ESTRATEGIA: emplear el campo 'uuid' y corelacionarlo con su versión original en inglés.

In [ ]:
def getFirstToken(string, char):
    t = string.index(char,0)
    return string[0:t]

In [ ]:
def searchByToken(df, token):
    i = 0
    R = []
    for i in range(len(df)):
        if (df[i] == token):
            R.append(i)
            return R 
    return R

In [ ]:
uuid = list(cards['uuid'])
uuid_first_token = [getFirstToken(x, '-') for x in uuid]
print(uuid[0])
print(uuid_first_token[0])

> Una opción para encontrar mas rápido es construir un bag of words y vectorizar el resultado, sin embargo, generar lo anterior conlleva demasiada memoria (+80Gb). Se opta por trabajar consultas por términos. El parámetro que es global y es independiente del idioma es el **uuid**. Para reducir el tiempo de busqueda, solo se usará el primer token del uuid.

In [ ]:
def unicidad_uuid():
    start = time.time()
    not_unique= []
    for i in range( len(uuid_first_token)):
        a = getFirstToken(uuid[i],'-')
        st = searchByToken(uuid_first_token, a)
        if(len(st)>1):
            not_unique.append(i)
    if (len(not_unique)>0):
        print('NO HAY UNICIDAD, para un primer token de uuid hay más de un resultado')
    else:
        print('UNICIDAD, se cumple unicidad de tokens')
        
    end = time.time()
    print ('time:', end - start)
    return not_unique

#unico = unicidad_uuid()

Se probó si el primer token del uuid es único. Los resultados lo comprueban, por lo tanto, buscamos solamente con la lista de **uuid_first_token** (primer token de cada uuid del universo total de cartas)

In [ ]:
def get_uuid_from_nan_values(LIST):
    type_nan_uuid = []
    type_nan_id = []
    for i in range(len(LIST)):
        if (type(LIST[i]) == float):
            type_nan_id.append(i)
            _token = getFirstToken(esp.iloc[i].uuid,'-')
            type_nan_uuid.append(_token)
    
    return type_nan_id, type_nan_uuid

In [ ]:
LIST_TYPES = list(esp['type'])
types_id, types_uuid = get_uuid_from_nan_values(LIST_TYPES)

In [ ]:
print("CARDS with 'type' = NaN:",len(types_uuid))
_esp = esp.iloc[types_id[0]]
print(_esp)
print(_esp['name'], _esp['id'], _esp['uuid'])

_esp_id = _esp['index']
_card = cards.iloc[_esp_id]
print(_card['name'], _card['id'], _card['uuid'],' <- NOT MATCH with id/index values!')

r = searchByToken(uuid_first_token, getFirstToken(_esp['uuid'],'-'))[0]
print('\n** Real id card in english:', r)

_card = cards.iloc[r]
print(_card['name'], _card['id'], _card['uuid'],' <- MATCH by uuid!')

> Hay 7214 cartas en español sin un type definido.

Analizando la primera carta en español con type = NaN, probamos si con el id o el index dado en el dataframe podemos llegar a la carta en inglés, sin embargo, los uuid no coinciden.

Haciendo match via uuid es la opción.

Linkear types nan a inglés.

In [ ]:
def match_types_Spanish(attribute, list_ids, uuids): 
    start = time.time()
    for i in range(len(list_ids)):
        index_esp = list_ids[i]
        first_token = uuids[i]
        index_eng = searchByToken(uuid_first_token, first_token)[0]
        _card = cards.iloc[index_eng]
        esp[attribute].iloc[index_esp] = _card[attribute]
    end = time.time()
    print ('time:', end - start)

In [ ]:
match_types_Spanish('type', types_id, types_uuid)

In [ ]:
_esp = esp.iloc[types_id[0]]
print(_esp)

In [ ]:
esp.info()

### ya tenemos linkeados los types en español.

## Objetivo 2:
Ahora completaremos los valores de text y flavorText

In [ ]:
for attribute in ['text', 'flavorText']:
    print('MATCHING ', attribute)
    LIST_ATTR = list(esp[attribute])
    ids, uuids = get_uuid_from_nan_values(LIST_ATTR)
    match_types_Spanish(attribute, ids, uuids)

In [ ]:
esp.info()

In [ ]:
def nullableTest(attribute):
    e = list(esp[attribute])[0:100]
    for i in range(len(e)):
        if (type(e[i]) == float):
            _test = esp.iloc[i]
            r = searchByToken(uuid_first_token, getFirstToken(_test['uuid'],'-'))[0]
            _card = cards.iloc[r]
            print(i, e[i], _test['uuid'],' english->' ,_card['id'], _card[attribute], _card['uuid'])

Se comprueba si el algoritmo funcionó, probando por cada atributo nulo en el dataframe si también lo es en el dataframe inglés.

In [ ]:
for attribute in ['text', 'flavorText']:
    print('Testing: ', attribute)
    print('\nid_lang, attribute, uuid_lang, || english_id, english_atribute, english_uuid')
    nullableTest(attribute)

In [ ]:
esp.to_csv('./output/spanish_cards.csv',index = False, header=True)